In [1]:
from sklearn.preprocessing import StandardScaler # scales variables to be mean=0, sd=1
from sklearn.pipeline import Pipeline # in case multiple things are performed at the same time
from sklearn.ensemble import GradientBoostingRegressor # Model 3
from sklearn.ensemble import RandomForestRegressor # Model 2
from sklearn.linear_model import LassoCV # Model 1
from sklearn.linear_model import Lasso # To save time and not do the Cross-Validation every time
from sklearn.svm import LinearSVR # Model 4
from sklearn.model_selection import GridSearchCV # Hypertuning of parameters
from sklearn.ensemble import VotingRegressor # Ensemble estimator
import pandas as pd
import numpy as np
import time

In [2]:
def CombineAttributes(data, var_list):
    
    for i in var_list:
        for j in var_list:
            
            if i == j:
                name = str(i)+ '_square'
                data[name] = data.loc[:, i] * data.loc[:, i]

            elif i <=j:
                name =  str(i)+ '_' +str(j)
                data[name] = data.loc[:, i] * data.loc[:, j]
            else:
                pass

    return data

In [509]:
dataframe = pd.read_excel(r'C:\Users\mariu\Documents\Project Local Elections\bayern_complete.xlsx')
dataframe.head()

,city_id,debt,non_public_debt,corp_debt,hospital_debt,year,county_id,no_krankenhauser,no_beds_avg,total_unemployed,...,total_german_female,total_german_male,total_pop,total_pop_county,votes,youth_list,ödp,con_list,li_li,franken
0,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
1,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
2,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
3,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
4,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0


In [510]:
dataframe_test = dataframe[dataframe['year'] == 2020].reset_index().drop(['index'], axis=1)
dataframe_train = dataframe[(dataframe['year'] == 2014) | (dataframe['year'] == 2008)
                           ].dropna(subset = ['incumbent', 'asyl', 'cost_o_liv', 'capital_crimes', 'seats_pre'
                                             ]).reset_index().drop(['index'], axis=1)
dataframe_test.head()

,city_id,debt,non_public_debt,corp_debt,hospital_debt,year,county_id,no_krankenhauser,no_beds_avg,total_unemployed,...,total_german_female,total_german_male,total_pop,total_pop_county,votes,youth_list,ödp,con_list,li_li,franken
0,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
1,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
2,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
3,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0
4,9162,680371.0,680371.0,1298244.0,0.0,2020,9162,54.0,11200.0,32465.0,...,563473,525802,1471508,1471508.0,NaN,0,0,0,0,0


In [511]:
X_names = ['female_total_x', 'cost_o_liv', 'asyl', 'BP_votes', 'CSU_votes', 'SPD_votes', 
           'Linke_votes', 'FDP_votes', 'Grüne_votes', 'AfD', 'BP', 'CSU', 'FDP', 'con_list', 'li_li', 'franken',
           'FW', 'Grüne', 'Linke', 'SPD', 'age0_5', 'age15_18', 'age18_24', 'age25_30', 'age30_40', 
           'age40_50', 'age50_65', 'age65plus', 'age6_14', 'anteil_einkommensteuer', 'anteil_umsatzsteuer',
           'capital_crimes', 'doctor', 'gewerbesteuerumlage', 'importance_climate', 'importance_social_sec', 'incumbent',
           'istaufkommen', 'mayor', 'muslim', 'new_list', 'seats_pre', 'non_muslim_migrant', 'place_list', 'globalisation',
           'politician', 'royal', 'search_result_stan', 'steuern_total', 'students',
           'total_foreigner', 'total_pop', 'total_unemployed', 'unemployed_longtime', 'youth_list', 'ödp']

to_scale = ['female_total_x', 'cost_o_liv', 'asyl',  'BP_votes', 'CSU_votes', 'SPD_votes', 'Linke_votes', 'FDP_votes',
            'gewerbesteuerumlage', 'Grüne_votes', 'age0_5', 'age15_18', 'age18_24', 'age25_30', 'age30_40',
            'age40_50', 'age50_65', 'age65plus', 'age6_14', 'capital_crimes',
            'istaufkommen', 'students', 'total_foreigner', 'total_pop', 'total_unemployed', 'unemployed_longtime', 
            'steuern_total', 'anteil_einkommensteuer', 'anteil_umsatzsteuer']

y_train = dataframe_train['votes']

X_train = dataframe_train[X_names]
X_test = dataframe_test[X_names]

X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled_2 = StandardScaler().fit_transform(X_train[to_scale])
X_test_scaled_2 = StandardScaler().fit(X_train[to_scale]).transform(dataframe_test[to_scale])

X_train_scaled[to_scale] = X_train_scaled_2
X_test_scaled[to_scale] = X_test_scaled_2

In [501]:
X_test_scaled.columns[X_test_scaled.isna().any()].tolist()

[]

In [500]:
X_train_scaled.columns[X_test_scaled.isna().any()].tolist()

[]

In [456]:
param_test1 = {'n_estimators':range(20,200,20)}
param_test2 = {'max_depth':range(5,18,3), 'min_samples_split':range(200,1001,100)}
param_test3 = {'min_samples_leaf': range(10, 200, 30) }

t1 = time.time()
gradient_boosting_cv = GridSearchCV(estimator = GradientBoostingRegressor
                                    (random_state=1, min_samples_leaf = 40, 
                                    n_estimators=160, loss = 'ls', learning_rate = 0.1, max_features = 'sqrt',
                                    criterion = 'mse', verbose = 0, max_depth = 8, min_samples_split = 400)
                                    , param_grid = param_test3, cv=10, error_score='raise', verbose = 1, n_jobs = -1)
t_grad_cv = time.time() - t1

t2 = time.time()
gradient_boosting_cv.fit(X_train_scaled, y_train)
t_grad_reg = time.time() - t2

gradient_boosting_cv.best_params_, gradient_boosting_cv.best_score_, t1, t2, gradient_boosting_cv.score(X_train_scaled,y_train)

Fitting 10 folds for each of 7 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:   18.0s finished


({'min_samples_leaf': 40},
 0.7486810347221352,
 1583079457.6754127,
 1583079457.6754127,
 0.9888597134319791)

In [457]:
grad_boost = GradientBoostingRegressor(random_state=1, min_samples_leaf = 40, 
                                    n_estimators=160, loss = 'ls', learning_rate = 0.1, max_features = 'sqrt',
                                    criterion = 'mse', verbose = 0, max_depth = 8, min_samples_split = 400)

grad_boost.fit(X_train_scaled, y_train)
grad_boost.score(X_train_scaled, y_train), grad_boost.feature_importances_

(0.9888597134319791,
 array([4.95801927e-02, 1.11302414e-02, 6.91078496e-02, 2.45772623e-02,
        2.41354035e-02, 9.11008957e-03, 6.37995574e-03, 6.13338239e-02,
        4.77025848e-03, 3.30985395e-05, 1.63233439e-03, 3.52638353e-02,
        2.86312916e-04, 2.95265281e-03, 1.65914355e-03, 1.38938510e-05,
        1.19426943e-03, 4.75873195e-03, 7.07379146e-05, 7.64964987e-02,
        8.54704431e-03, 9.60052841e-03, 8.87803889e-03, 3.91422583e-03,
        3.52470232e-04, 4.22439317e-03, 2.86019963e-03, 3.71810047e-02,
        2.62262131e-03, 5.71715878e-03, 8.96838688e-03, 6.30446512e-03,
        9.04308714e-05, 7.69671877e-03, 0.00000000e+00, 0.00000000e+00,
        2.52909312e-02, 1.42806602e-02, 0.00000000e+00, 3.01981840e-06,
        1.59002986e-04, 3.64384588e-01, 5.22264140e-06, 8.67274742e-03,
        0.00000000e+00, 0.00000000e+00, 1.50067535e-06, 3.69690315e-04,
        2.82442853e-02, 4.11078037e-02, 5.58979831e-03, 4.45061986e-03,
        3.17098846e-03, 1.27209324e-02, 1.1

In [459]:
param_grid_1 = {
    #'max_depth': range(10,100,10),
    #'min_samples_leaf': [4, 8, 12, 16, 20],
    'min_samples_split': [10,20,30,40,50],
    'n_estimators': range(10,100,10),
}

random_forest = GridSearchCV(estimator = RandomForestRegressor()
                             , param_grid = param_grid_1, cv = 10, verbose = 1, n_jobs = -1)
t2 = time.time()
random_forest.fit(X_train_scaled, y_train)
t_random_forest = time.time() - t2


random_forest.score(X_train_scaled,y_train), random_forest.best_params_

Fitting 10 folds for each of 45 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  7.7min
[Parallel(n_jobs=-1)]: Done 450 out of 450 | elapsed:  8.1min finished


(0.9969922010643119, {'min_samples_split': 10, 'n_estimators': 70})

In [462]:
rf = RandomForestRegressor(max_depth= 40, min_samples_leaf= 4, min_samples_split= 10, n_estimators = 70)
rf.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=70,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [460]:
Cs = [1, 2, 5, 8, 10, 20]
param_grid_svr = {'C': Cs, 'max_iter': range(1000, 10000, 1000)}

svr_cv = GridSearchCV(estimator = LinearSVR(), param_grid = param_grid_svr, cv=10, n_jobs = -1, verbose = 1)
svr_cv.fit(X_train_scaled, y_train)
svr_cv.best_params_, svr_cv.score(X_train_scaled, y_train)

Fitting 10 folds for each of 54 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 335 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed:   37.9s finished


({'C': 20, 'max_iter': 3000}, 0.31431066413864406)

In [463]:
svr = LinearSVR(C=20, max_iter = 3000)
svr.fit(X_train_scaled, y_train)

LinearSVR(C=20, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=3000,
          random_state=None, tol=0.0001, verbose=0)

In [473]:
param_grid_1 = {
    'alpha': np.logspace(0,3,50)
}

#lasso_cv = GridSearchCV(estimator = Lasso(), param_grid = param_grid_1, cv = 10, n_jobs= -1, verbose = 1)
#lasso_cv.fit(X_train_scaled, y_train)
lasso_cv.best_params_, lasso_cv.score(X_train_scaled, y_train)

({'alpha': 120.67926406393289}, 0.6451662166927664)

In [474]:
lasso = Lasso(alpha = 120.67926406393289)

In [486]:
ensemble_regression = VotingRegressor([('random_forest', rf), ('gradient_boosting', grad_boost), ('SVR', svr),
                                       ('Lasso', lasso)])

param_ensemble = {'weights': [(1, 1, 2, 1), (1, 2, 1, 1), (2, 1, 1, 1), (1, 1, 1, 2), (2, 2, 1, 1), (2, 1, 2, 1), 
                             (2, 1, 1, 2), (1, 2, 2, 1), (1, 2, 1, 2), (1, 1, 2, 2), (2, 2, 2, 1), (2, 2, 1, 2), (1, 2, 2, 2),
                             (1, 1, 1, 1), (1, 1, 1, 0), (1, 1, 0, 1), (1, 0, 1, 1), (0, 1, 1, 1), (1, 1, 0, 0), (1, 0, 1, 0),
                             (1, 0, 0, 1), (0, 1, 1, 0), (0, 1, 0, 1), (0, 0, 1, 1), (2, 2, 1, 0), (2, 2, 0, 1), (2, 1, 0, 2),
                             (2, 0, 1, 2), (1, 0, 2, 2), (0, 1, 2, 2), (0, 2, 1, 2), (1, 2, 0, 2), (2, 1, 2, 0), (2, 0, 2, 1),
                             (1, 2, 2, 0), (0, 2, 2, 1), (2, 1, 1, 0), (2, 1, 0, 1), (2, 1, 0, 0), (2, 0, 1, 1), (2, 0, 1, 0),
                             (2, 0, 0, 1), (1, 2, 1, 0), (1, 2, 0, 1), (0, 2, 1, 0), (1, 2, 0, 0), (0, 2, 1, 1), (0, 0, 2, 1),
                             (0, 1, 2, 1), (0, 1, 2, 1), (1, 0, 2, 1), (1, 1, 2, 0), (1, 0, 2, 0), (0, 1, 2, 0), (0, 0, 1, 2),
                             (0, 1, 0, 2), (1, 0, 0, 2), (1, 1, 0, 2), (1, 0, 1, 2), (0, 1, 1, 2),
                            (3,3,2,2),(3,2,3,2),(3,2,2,3),(2,3,3,2),(2,2,3,3),(2,3,3,2),
                              (3,2,3,3),(3,3,2,3),(2,3,3,3),(3,3,3,2),(2,2,2,3),(2,2,3,2),(2,3,2,2),(3,2,2,2),
                              (3,2,1,1),(3,2,1,2),(3,2,1,3),(3,2,1,0),(3,2,2,0),(3,2,2,1),(3,2,3,0),(3,2,3,1),
                            (3,3,1,0),(3,3,1,1),(3,3,1,2),(3,3,1,3),(3,3,0,1),(3,3,0,2),(3,3,0,3,),(3,3,0,0,),
                              (3,1,0,0),(3,1,0,1),(3,1,0,2,),(3,1,0,3),(3,1,1,0),(3,1,1,1),(3,1,1,2),(3,1,1,3),
                              (3,1,2,0),(3,1,2,1),(3,1,2,2),(3,1,2,2),(3,1,2,3),(3,1,3,0),(3,1,2,1),(3,1,2,2),(3,1,2,3),
                              (1,3,0,1),(1,3,0,1),(1,3,0,2),(1,3,0,3),(1,3,1,0),(1,3,1,1),(1,3,1,2),(1,3,1,3),(1,3,0,0),
                              (0,3,0,1),(0,3,0,1),(0,3,0,2),(0,3,0,3),(0,3,1,0),(0,3,1,1),(0,3,1,2),(0,3,1,3),(0,3,0,0),
                                (2,3,0,1),(2,3,0,1),(2,3,0,2),(2,3,0,3),(2,3,1,0),(2,3,1,1),(2,3,1,2),(2,3,1,3),(2,3,0,0)
                             ]}
ensemble_cv = GridSearchCV(estimator = ensemble_regression, param_grid = param_ensemble , cv=10, n_jobs = 4, verbose = 2)
ensemble_cv.fit(X_train_scaled, y_train)

ensemble_cv.best_params_, ensemble_cv.score(X_train_scaled, y_train)

Fitting 10 folds for each of 134 candidates, totalling 1340 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:  6.4min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed: 15.0min
[Parallel(n_jobs=4)]: Done 640 tasks      | elapsed: 27.1min
[Parallel(n_jobs=4)]: Done 1005 tasks      | elapsed: 42.9min
[Parallel(n_jobs=4)]: Done 1340 out of 1340 | elapsed: 57.6min finished
C:\Users\mariu\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
C:\Users\mariu\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40715425848.513306, tolerance: 374983097

({'weights': (2, 3, 1, 0)}, 0.972233323177091)

In [488]:
voting_reg = VotingRegressor([('random_forest', rf), ('gradient_boosting', grad_boost), ('SVR', svr),
                             ('Lasso', lasso)], weights = (2,3,1,0))
voting_reg.fit(X_train_scaled, y_train)

C:\Users\mariu\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 40715425848.513306, tolerance: 374983097.2762891
  positive)


VotingRegressor(estimators=[('random_forest',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=40,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=4,
                                                   min_samples_split=10,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=70, n_jobs=None,
                                                   oob_score=False,
                                                   random_state=None, verbose=0,
 

In [269]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score


def display_score(reg, mse_reg, r2):
    
    """ Insert explanation """
    
    reg_rmse = np.sqrt(-mse_reg)
    
    print('MSE                                  ')
    print('Scores:', reg_rmse, reg)
    print('Mean:', reg_rmse.mean(), reg)
    print('standard Deviation:', reg_rmse.std(), reg)
    print('R2                                   ')
    print('Scores:', r2, reg)
    print('Mean:', r2.mean(), reg)
    print('standard Deviation:', r2.std(), reg)

In [489]:
""" Gradient Boosting"""

mse_gb_train = cross_val_score(grad_boost, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_gb_train = cross_val_score(grad_boost, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("Gradient Boosting", mse_gb_train, r2_gb_train)

MSE                                  
Scores: [35699.7279224   9924.39165249  1056.2382968   1055.93830649
  1153.0728015   1189.04014962  1417.465078    4109.10609021
  3060.48190882  1972.26132515] Gradient Boosting
Mean: 6063.772353147104 Gradient Boosting
standard Deviation: 10209.775387656213 Gradient Boosting
R2                                   
Scores: [0.36161522 0.82667465 0.63021911 0.75572338 0.76112308 0.71588
 0.91728136 0.953178   0.6569245  0.90861172] Gradient Boosting
Mean: 0.748723100635542 Gradient Boosting
standard Deviation: 0.16596946323585168 Gradient Boosting


In [490]:
mse_rf_train = cross_val_score(rf, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_rf_train = cross_val_score(rf, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("RF", mse_rf_train, r2_rf_train)

MSE                                  
Scores: [31987.26891214  3068.39214463  1019.52956742  1046.70566063
  1138.44455437  1246.06035118  1894.69031107  9942.56107227
  4634.9067801   2796.66064093] RF
Mean: 5877.521999472223 RF
standard Deviation: 9076.78111226937 RF
R2                                   
Scores: [0.48813953 0.98175397 0.65747852 0.75855486 0.76472332 0.6961416
 0.86878173 0.68351213 0.23605764 0.79351469] RF
Mean: 0.6928657989939188 RF
standard Deviation: 0.19645573725496213 RF


In [491]:
mse_svr_train = cross_val_score(svr, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_svr_train = cross_val_score(svr, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("SVR", mse_svr_train, r2_svr_train)

MSE                                  
Scores: [44000.46903095 19511.82324387  1363.03773341  1820.18018144
  1614.7226798   1727.45093587  2677.98031622 15061.74830771
  3101.15922472  4028.98651612] SVR
Mean: 9490.755817011563 SVR
standard Deviation: 12987.1019354942 SVR
R2                                   
Scores: [0.03075119 0.33077623 0.38330371 0.27304863 0.53341854 0.39798198
 0.70486848 0.37259949 0.64685633 0.61816216] SVR
Mean: 0.4291766735673198 SVR
standard Deviation: 0.19210730994356404 SVR


In [492]:
mse_ee_train = cross_val_score(voting_reg, X_train_scaled, y_train, scoring = 'neg_mean_squared_error', cv = 10, n_jobs = -1)
r2_ee_train = cross_val_score(voting_reg, X_train_scaled, y_train, scoring = 'r2', cv = 10, n_jobs = -1)
display_score("Ensemble", mse_ee_train, r2_ee_train)

# -> Voting regression with weight combination 2,3,1,0 appears to be the most credible estimator
# Let's check out the redictions

MSE                                  
Scores: [35206.46481863  5858.11217059   958.47627866  1033.54942016
  1123.46423107  1167.70842275  1405.18046043  4378.96814244
  3066.6110851   2496.20653628] Ensemble
Mean: 5669.474156611736 Ensemble
standard Deviation: 9968.856162376123 Ensemble
R2                                   
Scores: [0.38142106 0.93901266 0.69636409 0.77068375 0.77548134 0.72471874
 0.91830228 0.943298   0.65973399 0.85191297] Ensemble
Mean: 0.7660928873165962 Ensemble
standard Deviation: 0.1605625925606985 Ensemble


In [512]:
voting_reg_pred = voting_reg.predict(X_test_scaled)

voting_pred = pd.DataFrame({'Prediction': voting_reg_pred, 'city': dataframe_test['city'],
                                      'party': dataframe_test['party'], 'last_name': dataframe_test['last_name'], 'city_id': dataframe_test['city_id'],
                                     'county_id': dataframe_test['county_id'], 'first_name': dataframe_test['first_name'],
                                      'place_list': dataframe_test['place_list'], 'seats_pre': dataframe_test['seats_pre']})

voting_pred.to_excel(r'C:\Users\mariu\Documents\Project Local Elections\forecast_candidates.xlsx')

In [494]:
list_coefficients = pd.DataFrame({'Variable': X_train.columns, 'Coefficients RF':rf.feature_importances_,
                                  'Features SVR': svr.coef_, 'GB Features': grad_boost.feature_importances_
                                 })
list_coefficients

,Variable,Coefficients RF,Features SVR,GB Features
0,female_total_x,2.247819e-02,620.512130,0.049580
1,cost_o_liv,3.250013e-02,-88.178196,0.011130
2,asyl,1.362492e-02,-35.359785,0.069108
3,BP_votes,2.412623e-02,-947.807645,0.024577
4,CSU_votes,2.602985e-02,605.651121,0.024135
5,SPD_votes,1.547213e-02,658.207737,0.009110
6,Linke_votes,3.081726e-02,1844.647738,0.006380
7,FDP_votes,2.194367e-02,-1010.931102,0.061334
8,Grüne_votes,2.618392e-02,-511.385777,0.004770
9,AfD,5.855405e-05,552.551204,0.000033


In [508]:
data_table_2 = dataframe_test.groupby(['city', 'year', 'party'], as_index = False).max()
data_table_2.to_excel(r'C:\Users\mariu\Documents\Project Local Elections\data_table_2.xlsx')

In [534]:
data_2014 = dataframe_train[dataframe_train['year'] ==2014]
data_party_2014 = data_2014.groupby(['party', 'city'], as_index = False).mean()
len(data_party_2014[data_party_2014['party']=='ödp'])

23

In [537]:
len(data_party_2014[(data_party_2014['party'] != 'Linke')&(data_party_2014['party'] != 'SPD')&(data_party_2014['party'] != 'CSU')&
                    (data_party_2014['party'] != 'Grüne')&(data_party_2014['party'] != 'FW')&(data_party_2014['party'] != 'BP')&(data_party_2014['party'] != 'BIA')&
                    (data_party_2014['party'] != 'FDP')&(data_party_2014['party'] != 'AfD')&(data_party_2014['party'] != 'ödp')&(data_party_2014['party'] != 'Partei')])

70